In [1]:
import json
import os
import pandas as pd
import numpy as np
from buckaroo.buckaroo_widget import BuckarooWidget
import requests
from multiprocessing.pool import ThreadPool as Pool

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [10]:
!#mkdir -p  ~/code/closest-bike/raw-json

In [2]:
base_dir = "/Users/paddy/code/closest-bike/raw-json"

In [3]:
gbfs_df = pd.read_csv("/Users/paddy/Downloads/gbfs-systems.csv")
BuckarooWidget(gbfs_df)

BuckarooWidget(commandConfig={'argspecs': {'dropcol': [None], 'to_datetime': [None], 'safeint': [None], 'filln…

In [13]:
gbfs_df['Auto-Discovery URL'][0]

'https://dubai.publicbikesystem.net/customer/gbfs/v2/gbfs.json'

In [14]:
ab = requests.get('https://dubai.publicbikesystem.net/customer/gbfs/v2/en/station_information')
bc = json.loads(ab.text)
BuckarooWidget(pd.DataFrame(bc['data']['stations']))

BuckarooWidget(commandConfig={'argspecs': {'dropcol': [None], 'to_datetime': [None], 'safeint': [None], 'filln…

In [15]:
def get_system_data(index, url):
    try:
        raw_data = json.loads(requests.get(url).text)
        actual_data = raw_data['data']['en']['feeds']
        open("%s/feeds-%d.json" % (base_dir, index), "w").write(json.dumps(actual_data))
    except Exception as e:
        print(index, e)

In [16]:
def f(arg):
    print("arg", arg)
    get_system_data(*arg)
with Pool(30) as p:
    #print(p.map(f, [1, 2, 3]))
    p.map(f, enumerate(gbfs_df['Auto-Discovery URL']))

argarg (7, 'https://gbfs.nextbike.net/maps/gbfs/v2/nextbike_ka/gbfs.json')
 (0, 'https://dubai.publicbikesystem.net/customer/gbfs/v2/gbfs.json')
arg (14, 'https://mds-global-adl-cbd.neuron-mobility.com/gbfs/2/')
arg (21, 'https://mds-global-hba.neuron-mobility.com/gbfs/2/')
arg (28, 'https://aruba.publicbikesystem.net/customer/gbfs/v2/gbfs.json')
arg (35, 'https://stables.donkey.bike/api/public/gbfs/2/donkey_gh/gbfs.json')
arg (42, 'https://data.lime.bike/api/partners/v2/gbfs/antwerp/gbfs.json')
arg (49, 'https://rec.publicbikesystem.net/customer/gbfs/v2/gbfs.json')
arg (56, 'https://saguenay.publicbikesystem.net/customer/gbfs/v2/gbfs.json')
arg (63, 'https://gbfs.hopr.city/api/gbfs/22/')
arg (70, 'https://mds-global-ard.neuron-mobility.com/gbfs/2/')
arg (77, 'https://mds-global-yve.neuron-mobility.com/gbfs/2/')
arg (84, 'https://gbfs.hopr.city/api/gbfs/13')
arg (91, 'https://mds.bird.co/gbfs/v2/public/zurich/gbfs.json')
arg (98, 'https://stables.donkey.bike/api/public/gbfs/2/donkey_yv

In [19]:
!ls ~/code/closest-bike/raw-json/ | grep feed | wc -l

     741


In [20]:
v2_df = gbfs_df[gbfs_df['Auto-Discovery URL'].apply(lambda url: "v2" in url)]

In [34]:
def get_station_information_url(idx):
    feed = json.loads(open("%s/feeds-%d.json" % (base_dir, idx)).read())
    objs = [obj for obj in feed if obj['name'] == 'station_information']
    try:
        return objs[0]['url']
    except:
        print("problem with", idx)
        return ""

In [33]:
#get_station_information_url(806)

'https://gbfs.spin.pm/api/gbfs/v2_3/white_center/station_information'

In [28]:
raw_data = json.loads(requests.get("https://gbfs.nextbike.net/maps/gbfs/v2/nextbike_su/en/station_information.json").text)
actual_data = raw_data['data']
actual_data['stations'][:2]

[{'station_id': '6364500',
  'name': 'Manor Park Residences',
  'short_name': '8779',
  'lat': 51.237795,
  'lon': -0.61335,
  'region_id': '485',
  'rental_uris': {'android': 'https://app.nextbike.net/station?id=6364500',
   'ios': 'https://app.nextbike.net/station?id=6364500',
   'web': 'https://nxtb.it/p/6364500'},
  'is_virtual_station': False,
  'capacity': 14},
 {'station_id': '6364524',
  'name': 'Surrey Technology Centre',
  'short_name': '8780',
  'lat': 51.239579,
  'lon': -0.611192,
  'region_id': '485',
  'rental_uris': {'android': 'https://app.nextbike.net/station?id=6364524',
   'ios': 'https://app.nextbike.net/station?id=6364524',
   'web': 'https://nxtb.it/p/6364524'},
  'is_virtual_station': False,
  'capacity': 14}]

In [35]:
def get_station_data(index, url):
    try:
        raw_data = json.loads(requests.get(url).text)
        actual_data = raw_data['data']['stations']
        open("%s/stations-%d.json" % (base_dir, index), "w").write(json.dumps(actual_data))
    except Exception as e:
        print(index, e)
    

In [37]:
def f(idx):
    try:
        url = get_station_information_url(idx)
        get_station_data(idx, url)
    except:
        print("problem with idx")
with Pool(30) as p:
    #print(p.map(f, [1, 2, 3]))
    p.map(f, v2_df.index)

problem with 38
38 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 39
39 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with idx
problem with 40
40 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 41
41 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 222
222 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 223
223 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 224
224 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 225
225 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with idx
problem with 185
185 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 281
281 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 282
282 Invalid URL '': No scheme supplied. Perhaps you meant https://?
problem with 283
283 Invalid URL '': No

In [38]:
!ls ~/code/closest-bike/raw-json/station*  |wc -l

     422


In [39]:
pd.read_json("/Users/paddy/code/closest-bike/raw-json/stations-105.json")[:3]

,station_id,name,physical_configuration,lat,lon,altitude,address,cross_street,post_code,capacity,is_charging_station,rental_methods,is_virtual_station,groups,nearby_distance,_bluetooth_id,_ride_code_support,rental_uris,obcn,is_valet_station
0,1,001 - Centro Comercial Andino,REGULAR,4.665803,-74.052844,NaN,Calle 82 entre Carrera 11 y Carrera 12,EL RETIRO,1111.0,19,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",False,[],1000,121b,True,{},NaN,NaN
1,2,002 - CL 65 con KR 10,REGULAR,4.650789,-74.061453,0.0,Calle 65 entre Carrera 10 y Carrera 11,CHAPINERO NORTE,1111.0,7,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",False,[],1000,be17,True,{},,NaN
2,3,003 - CL 73 con KR 15,REGULAR,4.659926,-74.060431,0.0,Calle 73 entre Carrera 13 y Carrera 15,LA CABRERA,1111.0,7,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",False,[],1000,484d,True,{},,NaN


In [ ]:
print("base_dir", base_dir)

In [40]:

station_f_list = [f for f in os.listdir(base_dir)if "stations" in f]
dfs = [pd.read_json(base_dir + "/" + f)  for f in station_f_list]

In [41]:
all_dfs = pd.concat(dfs)

In [42]:
all_dfs['capacity'].sum()

205262.0

In [ ]:
BuckarooWidget(all_dfs)

In [ ]:
!ls ~/code/closest-bike/raw-json  | tail

In [ ]:
actual_data['stations'][:3]

In [ ]:
for idx in v2_df.index:
    try:
        url = get_station_information_url(idx)
        

In [ ]:
v2_df.index[:10]

In [ ]:
for idx, url  in enumerate(gbfs_df['Auto-Discovery URL']):
    print(idx)
    get_system_data(idx, url)

In [ ]:
def pull_auto_discovery(index_num, url):
    system_data = json.loads(requests.get(url).text)